In [1]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output

# --- Swiss Design CSS ---
style = """
<style>
    /* FONT & BOX-SIZING RESET */
    .swiss-wrapper {
        font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
    }
    .swiss-wrapper, .swiss-wrapper * {
        box-sizing: border-box !important;
    }

    /* MARGIN RESET for Widgets to prevent overflow with width: 100% */
    .swiss-wrapper .widget-hbox, 
    .swiss-wrapper .widget-vbox,
    .swiss-wrapper .widget-text,
    .swiss-wrapper .widget-dropdown,
    .swiss-wrapper .widget-input {
        margin-left: 0 !important;
        margin-right: 0 !important;
    }

    /* Header */
    .swiss-header {
        border-bottom: 4px solid #FF0000;
        padding-bottom: 20px;
        margin-bottom: 40px;
    }
    .swiss-header h1 {
        font-size: 48px;
        font-weight: 700;
        letter-spacing: -1px;
        margin: 0;
        text-transform: uppercase;
        color: #000;
    }
    .swiss-header p {
        font-size: 18px;
        font-weight: 300;
        margin-top: 5px;
        color: #666;
    }

    /* Table */
    .swiss-table table {
        width: 100%;
        border-collapse: collapse;
        margin-top: 10px;
        font-size: 16px;
    }
    .swiss-table th {
        text-align: center;
        border-bottom: 2px solid #000;
        padding: 10px;
        text-transform: uppercase;
        font-size: 12px;
        letter-spacing: 1px;
    }
    .swiss-table th:first-child {
        text-align: left;
    }
    .swiss-table td {
        text-align: center;
        border-bottom: 1px solid #eee;
        padding: 12px 10px;
    }
    .swiss-table td:first-child {
        text-align: left;
    }
    .swiss-table tr:hover td {
        background-color: #f9f9f9;
    }
</style>
"""
display(HTML(style))

# Emissionsfaktoren
EMISSION_FACTORS = {
    "Auto": 0.12,
    "Zug": 0.04,
    "Flug": 0.25,
    "Velo": 0.00
}

IS_VEHICLE_EMISSION = {
    "Auto": True,
    "Zug": False,
    "Flug": False,
    "Velo": False
}

# --- Widgets ---

# Fix: Using calc() to be super safe (99% instead of 100%) or relying on margin reset above
input_layout = widgets.Layout(width='100%')

transport_dropdown = widgets.Dropdown(
    options=['Auto', 'Zug', 'Flug', 'Velo'],
    value='Auto',
    description='Transport:',
    style={'description_width': '100px'},
    layout=input_layout
)

distance_input = widgets.Text(
    value='',
    placeholder='0.0',
    description='Distanz (km):',
    style={'description_width': '100px'},
    layout=input_layout,
    continuous_update=True
)

persons_input = widgets.IntText(
    value=1,
    description='Personen:',
    style={'description_width': '100px'},
    layout=input_layout,
    continuous_update=True
)

output_area = widgets.Output()

controls_box = widgets.VBox(
    [transport_dropdown, distance_input, persons_input],
    layout=widgets.Layout(
        margin='0 0 50px 0',   
        width='auto',          
        padding='0 20px 20px 0' 
    )
)

# --- Logik ---
def calculate_co2(*args):
    selected_mode = transport_dropdown.value
    try:
        dist_str = distance_input.value.replace(',', '.')
        dist = float(dist_str) if dist_str.strip() else 0.0
    except ValueError:
        return
        
    persons = max(1, persons_input.value)

    def get_per_person_co2(mode, distance, count):
        base = EMISSION_FACTORS.get(mode, 0)
        if IS_VEHICLE_EMISSION.get(mode, False):
            return (base * distance) / count
        return base * distance

    selected_co2 = get_per_person_co2(selected_mode, dist, persons)
    
    html_content = """
    <div class='swiss-table'>
        <table>
            <thead>
                <tr>
                    <th>Transportmittel</th>
                    <th>CO2 / Person (kg)</th>
                    <th>Unterschied</th>
                </tr>
            </thead>
            <tbody>
    """
    
    for mode in EMISSION_FACTORS.keys():
        co2 = get_per_person_co2(mode, dist, persons)
        
        if mode == selected_mode:
            row_style = "font-weight: bold; background-color: #fff0f0;"
            mode_display = f"<span style='color:#FF0000'>■</span> {mode}"
            diff_display = "—"
        else:
            row_style = ""
            mode_display = mode
            diff = co2 - selected_co2
            if diff > 0:
                diff_display = f"<span style='color:#FF0000'>+{diff:.2f}</span>"
            elif diff < 0:
                diff_display = f"<span style='color:#00AA00'>{diff:.2f}</span>"
            else:
                diff_display = "0.00"
                
        html_content += f"""
            <tr style='{row_style}'>
                <td>{mode_display}</td>
                <td>{co2:.2f}</td>
                <td>{diff_display}</td>
            </tr>
        """
        
    html_content += "</tbody></table></div>"
    
    with output_area:
        output_area.clear_output(wait=True)
        display(HTML(html_content))

transport_dropdown.observe(calculate_co2, names='value')
distance_input.observe(calculate_co2, names='value')
persons_input.observe(calculate_co2, names='value')

# --- Haupt-Layout ---
header_html = widgets.HTML(
    value="""
    <div class='swiss-header'>
        <h1>Carbon<span style='color:#FF0000'>.</span>CH</h1>
        <p>Mobilitäts-Tracker & Vergleich</p>
    </div>
    """
)

main_container = widgets.VBox(
    [header_html, controls_box, output_area],
    layout=widgets.Layout(
        width='640px', 
        margin='0 auto', 
        padding='30px',  
        border='1px solid #f0f0f0'
    )
)

# Apply wrapper class
main_container.add_class("swiss-wrapper")

# Additional Safe-Check: Remove margin from the inputs directly via layout to be double-sure
# (Though CSS should handle it via !important)
transport_dropdown.layout.margin = '0px'
distance_input.layout.margin = '0px'
persons_input.layout.margin = '0px'

calculate_co2() # Init
display(main_container)